In [ ]:
import pandas as pd

In [ ]:
#讀excel
df_orderData = pd.read_excel('oea_oeb_old.xlsx', sheet_name = 0)
df_orderData1 = pd.read_excel('oea_oeb_new.xlsx', sheet_name = 0)

In [ ]:
df = pd.DataFrame(df_orderData,columns=['OEB01','OEB03','OEB15','OEB16','OEA02','OEA14','TC_SFA104'])
df

In [ ]:
df["pk"] = df["OEB01"] + "_" + df["OEB03"].map(str)
df

In [ ]:
df1 = pd.DataFrame(df_orderData1,columns=['OEB01','OEB03','OEB15','OEB16','OEA02','OEA14','TC_SFA104'])
df1

In [ ]:
df1["pk"] = df1["OEB01"] + "_" + df1["OEB03"].map(str)
df1

In [ ]:
tmp = df.append(df1)
tmp.drop_duplicates(keep=False, inplace=True)
all_diff=pd.DataFrame(tmp)
all_diff

In [ ]:
last = all_diff.drop_duplicates(subset=['pk'],keep='last')
last

In [ ]:
first = all_diff.drop_duplicates(subset=['pk'],keep='first')
first

In [ ]:
pk_diff = all_diff.drop_duplicates(subset=['pk'],keep=False)
pk_diff

In [ ]:
new = last.append(pk_diff).drop_duplicates(subset=['pk'],keep=False)
new

In [ ]:
old = first.append(pk_diff).drop_duplicates(subset=['pk'],keep=False)
old

In [ ]:
merged_data = new.merge(old,on = ['pk','OEB01','OEB03','OEB15','OEA02','OEA14','TC_SFA104'], suffixes=('_new','_old') )
merged_data

In [ ]:
merged_data = new.merge(old,on = ['pk','OEB01','OEB03','OEB15','OEA02','OEA14'], suffixes=('_new','_old') )
merged_data

In [ ]:
merged_data.info()

In [ ]:
import datetime
# 抓完工日 = 預計下機日 + 10天
date_of_completion = merged_data['TC_SFA104_new'] + datetime.timedelta(days = 10)
# 當預計完工日超過約定交貨日時，表示可能來不及交貨
delayed_data = merged_data[date_of_completion > merged_data['OEB15']]
delayed_data

In [ ]:
delayed_data = delayed_data[delayed_data['OEB16_new'] != delayed_data['OEB16_old']].copy()
delayed_data

In [ ]:
#final = delayed_data1.loc[:, ['OEA14','OEA02','OEB01','OEB03','OEB15','OEB16_old','OEB16_new','TC_SFA104_old','TC_SFA104_new']]
final = delayed_data.loc[:, ['OEA14','OEA02','OEB01','OEB03','OEB15','OEB16_old','OEB16_new', 'TC_SFA104_new']]
final

In [ ]:
final.sort_values(['OEA14','OEB01'], inplace = True, ignore_index=True) # 依照業務編號、訂單編號做遞增排序
final.rename(columns={'OEA02': '訂單日期', 'OEA14': '業務編號', 'OEB01': '訂單號碼', 'OEB03': '項次', 'OEB15': '約定交貨日', 'OEB16_old': '原排定交貨日', 'OEB16_new': '新排定交貨日','TC_SFA104_new':"約定下機日"}, inplace=True)
final

In [ ]:
#dataframe轉html
df_html = final.to_html(escape=False,index=False, justify = 'center')

In [ ]:
# 當沒有符合判斷條件的資料時，回傳None
if final.empty == True:
    return None
# 回傳final的html格式
return df_html

In [ ]:
def read_file(file):
    with open(file,'r',encoding="utf-8") as f:
            content_list = f.read().split('\n')
    return content_list
emails = read_file('email.txt')
emails

In [ ]:

recipients = [emails[2]]

recipients 

In [ ]:
a=emails[2].split(', ')
a

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText#專門傳送正文
from email.mime.image import MIMEImage
import sys
import smtplib
import datetime

def SendMail():
    #print('start to send mail...')
    emails = read_file('email.txt') #讀email資料
    sender = emails[0] #step1:setup sender gmail,ex:"Fene1977@superrito.com"
    password = emails[1] #step2:setup sender gmail password
    recipients = emails[2].split(', ') #step3:setup recipients mail
    today = datetime.date.today()
    sub = today.strftime("%m/%d") + "訂單交期更改通知" #step4:setup your subject
    
    #單個收件人,ex:recipients = 'Fene1977@dayrep.com'  
    #多個收件人,ex:"Hinte1969@jourrapide.com,Fene1977@rhyta.com,Fene1977@teleworm.us"
    
    outer = MIMEMultipart()
    outer['From'] = sender #step:setup sender gmail
    outer['To'] = ','.join(recipients) #step:setup recipient mail
    #outer["Cc"]="cc mail" #step:setup cc mail
    outer['Subject'] = sub #step:setup your subject

    #設定純文字資訊
    plainText = "偵測到訂單排定交貨日更改，請確認以下訂單："
    msgText = MIMEText(plainText, 'plain', 'utf-8')
    outer.attach(msgText)
    #設定HTML資訊
    #htmlText = df_html #step7:edit your mail content
    #msgText = MIMEText(htmlText, 'html', 'utf-8')
    #outer.attach(msgText)

    mailBody = outer.as_string()
    #-----------------------------------------------------------------------
    
    # 寄送EMAIL
    try:
        with smtplib.SMTP(host="smtp.gmail.com", port="587") as s: #send webservice to gmail smtp socket
            s.ehlo()  # 驗證SMTP伺服器
            s.starttls()  # 建立加密傳輸
            s.login(sender, password)  # 登入寄件者gmail
            s.sendmail(sender, recipients,mailBody)  # 寄送郵件
            s.close()
        print("Email sent!")
    except:
        print("Unable to send the email. Error: ", sys.exc_info()[0])
        raise
       
SendMail()

In [ ]:
import os
def remove_file(file_name):
    try:
        os.remove(file_name)
    except OSError as e:
        print(e)
    else:
        print("File is deleted successfully")
file_name = file2
remove_file(file_name)

In [ ]:
def read_file(file):
    with open(file,'r',encoding="utf-8") as f:
            content_list = f.read().split('\n')
    return content_list
fileName = read_file('fileName.txt')
fileName[1]

In [ ]:
import shutil
def copy_file1_as_file2(file1, file2):
    try:
        shutil.copyfile(file1, file2)
        print("Copy %s as %s successfully"%(file1, file2))
    except OSError as err:
        print(err)

file1 = fileName[0]
file2 = fileName[2]

copy_file1_as_file2(file1, file2)

In [ ]:
import pandas as pd
#import numpy as np
from tqdm import tqdm
for i in tqdm(range(0,2), ncols = 100, desc ="Loading data..."): 
    df_orderData = pd.read_excel('oea_oeb_old.xlsx', sheet_name = 0)
    df_orderData = pd.read_excel('oea_oeb_new.xlsx', sheet_name = 0)
print("------Loading is completed ------")
df = pd.DataFrame(df_orderData,columns=['OEB01','OEB03','OEB15','OEB16','OEA02','OEA14','TC_SFA104'])   

